In [18]:
import os
import xarray as xr
import numpy as np
import pandas as pd
from glob import glob

# Create a CFDatetimeCoder instance
time_coder = xr.coders.CFDatetimeCoder(use_cftime=True)

##############################################################################
# 0) Helper to convert cftime => "YYYY-MM"
##############################################################################
def cftime_to_yearmonth(cf_time):
    """
    Convert a cftime object to a "YYYY-MM" string.
    If cf_time is float or NaN, return None.
    """
    if pd.isnull(cf_time) or isinstance(cf_time, float):
        return None
    return f"{cf_time.year:04d}-{cf_time.month:02d}"

##############################################################################
# 1) Read each monthly netCDF => Weighted monthly mean => [timestr, experiment, model, tas]
##############################################################################

all_rows = []
nc_files = glob("../data/cds/tas*.nc")
print(f"Found {len(nc_files)} monthly netCDF files...")

for fpath in nc_files:
    base = os.path.basename(fpath)
    print(f"\nReading file: {base}")
    ds = xr.open_dataset(fpath, decode_times=time_coder)

    exp = ds.attrs.get("experiment_id", "???")
    mod = ds.attrs.get("source_id", "???")
    print(f" => experiment={exp}, model={mod}")

    # If "height" is present, drop it
    if "height" in ds.variables:
        ds = ds.drop_vars("height")

    # Weighted monthly mean (convert K to °C)
    da_k = ds["tas"]  # shape: (time, lat, lon)
    da_c = da_k - 273.15
    weights = np.cos(np.deg2rad(da_k.lat))
    da_weighted = da_c.weighted(weights)
    da_mean = da_weighted.mean(dim=["lat", "lon"])  # 1D [time]

    # Convert to DataFrame
    df_temp = da_mean.to_dataframe(name="tas").reset_index()  # => [time, tas]
    # Convert cftime => "YYYY-MM"
    df_temp["timestr"] = df_temp["time"].apply(cftime_to_yearmonth)
    # Convert the string to a pandas datetime format (day will default to 1)
    df_temp["timestr"] = pd.to_datetime(df_temp["timestr"], format="%Y-%m", errors="coerce")

    # Add experiment & model
    df_temp["experiment"] = exp
    df_temp["model"] = mod

    print(f" => shape={df_temp.shape}, timestr example: {df_temp['timestr'].head(3).tolist()}")
    all_rows.append(df_temp[["timestr","experiment","model","tas"]])

##############################################################################
# 2) Concatenate => Group on [experiment, timestr] => Multi-model ensemble
##############################################################################
df_all = pd.concat(all_rows, ignore_index=True)
print("\nAll data shape =", df_all.shape)
print(df_all.head(10))

# By default, pandas .mean() ignores NaN values
df_ens = df_all.groupby(["experiment", "timestr"], as_index=False)["tas"].mean()
print("\nEnsemble groupby shape =", df_ens.shape)
print(df_ens.head())

##############################################################################
# 3) Historical vs Scenario splicing using pandas datetime
##############################################################################
def splice_hist_scenario(df_ens, scenario):
    """
    Input: df_ens with columns: [experiment, timestr, tas]
    timestr is pandas datetime.
    We combine 'historical' (1850-2014) + scenario (2015-2100).
    Returns a new DataFrame with columns: [timestr, year, month, tas].
    """
    # Create year and month columns
    df_ens["year"] = df_ens["timestr"].dt.year
    df_ens["month"] = df_ens["timestr"].dt.month

    df_hist = df_ens.query("experiment=='historical' and year>=1850 and year<=2014").copy()
    df_scen = df_ens.query("experiment==@scenario and year>=2015 and year<=2100").copy()

    df_comb = pd.concat([df_hist, df_scen], ignore_index=True)
    df_comb = df_comb.sort_values(["year", "month"]).reset_index(drop=True)
    return df_comb

df_126 = splice_hist_scenario(df_ens, "ssp126")
df_245 = splice_hist_scenario(df_ens, "ssp245")
df_585 = splice_hist_scenario(df_ens, "ssp585")

print("\nssp126 shape:", df_126.shape)
print("head:\n", df_126.head(10))

##############################################################################
# 4) Export dataframes (so you can import them in another file)
##############################################################################
export_folder = "../data/cds/"
df_126.to_csv(os.path.join(export_folder, "tas_ensemble_ssp126_monthly_1850_2100.csv"), index=False)
df_245.to_csv(os.path.join(export_folder, "tas_ensemble_ssp245_monthly_1850_2100.csv"), index=False)
df_585.to_csv(os.path.join(export_folder, "tas_ensemble_ssp585_monthly_1850_2100.csv"), index=False)

# Optionally, export as pickle:
df_126.to_pickle(os.path.join(export_folder, "tas_ensemble_ssp126_monthly_1850_2100.pkl"))
df_245.to_pickle(os.path.join(export_folder, "tas_ensemble_ssp245_monthly_1850_2100.pkl"))
df_585.to_pickle(os.path.join(export_folder, "tas_ensemble_ssp585_monthly_1850_2100.pkl"))

print("\nAll done. DataFrames exported as CSV and pickle files.")

Found 65 monthly netCDF files...

Reading file: tas_Amon_HadGEM3-GC31-LL_historical_r1i1p1f3_gn_185001-194912.nc
 => experiment=historical, model=HadGEM3-GC31-LL
 => shape=(1200, 5), timestr example: [Timestamp('1850-01-01 00:00:00'), Timestamp('1850-02-01 00:00:00'), Timestamp('1850-03-01 00:00:00')]

Reading file: tas_Amon_INM-CM5-0_ssp245_r1i1p1f1_gr1_201501-210012.nc
 => experiment=ssp245, model=INM-CM5-0
 => shape=(1032, 5), timestr example: [Timestamp('2015-01-01 00:00:00'), Timestamp('2015-02-01 00:00:00'), Timestamp('2015-03-01 00:00:00')]

Reading file: tas_Amon_MPI-ESM1-2-LR_ssp126_r1i1p1f1_gn_203501-205412.nc
 => experiment=ssp126, model=MPI-ESM1-2-LR
 => shape=(240, 5), timestr example: [Timestamp('2035-01-01 00:00:00'), Timestamp('2035-02-01 00:00:00'), Timestamp('2035-03-01 00:00:00')]

Reading file: tas_Amon_HadGEM3-GC31-LL_historical_r1i1p1f3_gn_195001-201412.nc
 => experiment=historical, model=HadGEM3-GC31-LL
 => shape=(780, 5), timestr example: [Timestamp('1950-01-01 

In [10]:
df_126

,experiment,timestr,tas,year,month
0,historical,1850-01,11.318312,1850,1
1,historical,1850-02,11.603154,1850,2
2,historical,1850-03,12.319791,1850,3
3,historical,1850-04,13.415382,1850,4
4,historical,1850-05,14.385354,1850,5
...,...,...,...,...,...
3007,ssp126,2100-08,17.352185,2100,8
3008,ssp126,2100-09,16.679186,2100,9
3009,ssp126,2100-10,15.687556,2100,10
3010,ssp126,2100-11,14.659475,2100,11


In [11]:
df_245

,experiment,timestr,tas,year,month
0,historical,1850-01,11.318312,1850,1
1,historical,1850-02,11.603154,1850,2
2,historical,1850-03,12.319791,1850,3
3,historical,1850-04,13.415382,1850,4
4,historical,1850-05,14.385354,1850,5
...,...,...,...,...,...
3007,ssp245,2100-08,18.700473,2100,8
3008,ssp245,2100-09,17.984499,2100,9
3009,ssp245,2100-10,16.994911,2100,10
3010,ssp245,2100-11,16.002053,2100,11


In [14]:
df_585

,experiment,timestr,tas,year,month
0,historical,1850-01,11.318312,1850,1
1,historical,1850-02,11.603154,1850,2
2,historical,1850-03,12.319791,1850,3
3,historical,1850-04,13.415382,1850,4
4,historical,1850-05,14.385354,1850,5
...,...,...,...,...,...
3007,ssp585,2100-08,20.885149,2100,8
3008,ssp585,2100-09,20.198120,2100,9
3009,ssp585,2100-10,19.151406,2100,10
3010,ssp585,2100-11,18.174843,2100,11


In [ ]:


# Open one sample file with decode_times=True
sample_file = "../data/cds/tas_Amon_HadGEM3-GC31-LL_historical_r1i1p1f3_gn_185001-194912.nc"
ds_sample = xr.open_dataset(sample_file, decode_times=True)

# Look at the time coordinate
time_values = ds_sample.time.values
print("Time coordinate values:", time_values)

# Compute the differences between time steps (in days)
time_diff = np.diff(time_values.astype('datetime64[D]'))
print("Time differences between entries:", time_diff)
print("Unique time differences (in days):", np.unique(time_diff))

In [ ]:
import xarray as xr
import numpy as np

# Define the sample file path
sample_file = "../data/cds/tas_Amon_HadGEM3-GC31-LL_historical_r1i1p1f3_gn_185001-194912.nc"

# Open the dataset with decoded times (this converts the time coordinate to cftime objects)
ds_sample = xr.open_dataset(sample_file, decode_times=True)

# Retrieve the time coordinate values
time_values = ds_sample.time.values
print("Time coordinate values:", time_values)

# Convert the time values to numpy datetime64 objects (using days) and compute differences
time_diff = np.diff(time_values.astype("datetime64[D]"))
print("Time differences between entries (in days):", time_diff)
print("Unique time differences (in days):", np.unique(time_diff))

In [ ]:
def cftime_to_monthly_string(cf_time):
    try:
        return f"{cf_time.year:04d}-{cf_time.month:02d}-15"
    except Exception as e:
        return None

# Test on your time coordinate:
date_strings = [cftime_to_monthly_string(t) for t in time_values]
print(date_strings[:10])

In [ ]:
ds_ens = xr.open_dataset("../data/cds/cmip6_ensemble_mean_monthly.nc", 
                           decode_times=True, use_cftime=True)
print(ds_ens.experiment.values)